## Installation des bibliothèques nécessaires
- **requests :** pour les requêtes HTTP 
- **beautifulsoup4 :** pour l'analyse HTML
- **PyDrive :** pour l'interaction avec Google Drive (optionnel)

In [1]:
!pip install requests
!pip install beautifulsoup4
!pip install PyDrive

## Web Scraping des Images

La cellule de code suivante définit une fonction `download_images` qui permet de télécharger automatiquement toutes les images d'une page web donnée. Cette technique, appelée web scraping, est utilisée ici pour collecter rapidement un grand nombre d'images pour notre jeu de données.

**Important :** 
- Assurez-vous d'avoir les droits nécessaires pour utiliser les images du site web choisi.
- Remplacez l'URL exemple par celle de votre choix.
- Ajustez le chemin du dossier de destination selon votre structure Google Drive.

In [2]:
# Importation des bibliothèques nécessaires
from bs4 import BeautifulSoup  # Pour l'analyse du contenu HTML
import requests  # Pour envoyer des requêtes HTTP
import os  # Pour les opérations sur les fichiers et répertoires
from google.colab import drive  # Pour monter Google Drive dans Colab

# Fonction pour télécharger des images depuis une URL
def download_images(url, folder_path):
    # Envoyer une requête GET à l'URL
    response = requests.get(url)
    # Analyser le contenu HTML de la réponse
    soup = BeautifulSoup(response.text, 'html.parser')
    # Trouver toutes les balises <img> (images) dans le HTML
    images = soup.find_all('img')

    # Vérifier si le chemin du dossier existe, sinon le créer
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for img in images:
        # Obtenir l'URL source de l'image
        img_url = img.get('src')
        # Vérifier si l'URL de l'image est relative (ne commence pas par http ou https)
        if not img_url.startswith(('http:', 'https:')):
            # Rendre l'URL absolue en la joignant avec l'URL de base
            img_url = requests.compat.urljoin(url, img_url)
        # Obtenir le nom de l'image à partir de l'URL
        img_name = os.path.basename(img_url)
        # Créer le chemin complet pour sauvegarder l'image
        full_path = os.path.join(folder_path, img_name)

        # Télécharger et sauvegarder l'image
        with open(full_path, 'wb') as file:
            img_response = requests.get(img_url)
            file.write(img_response.content)

    print(f'Téléchargé {len(images)} images dans {folder_path}')

# Exemple d'utilisation
download_images('https://www.behance.net/gallery/99930185/Mercedes-of-Mauritania', '/content/drive/MyDrive/RIMAI/behance')

## Renommage des Images

Pour répondre aux exigences du challenge, nous devons renommer nos images selon un format spécifique. La cellule de code ci-dessous contient une fonction `rename_images` qui s'occupe de cette tâche.

Le format de nommage est le suivant :
- Pour les images web : `web{numéro}_image_name.{extension}`
- Pour les photos de caméra : `cam{numéro}_image_name.{extension}`

**Note :** 
- Assurez-vous que les chemins vers vos dossiers d'images sont corrects.
- La fonction traitera tous les fichiers image (jpg, jpeg, png, gif) dans les dossiers spécifiés.

In [3]:
from google.colab import drive
import os

# Monter Google Drive dans l'environnement Colab
# Cela permet d'accéder aux fichiers stockés sur votre Google Drive
drive.mount('/content/drive')

# Définir les chemins vers les dossiers contenant les images
# Un dossier pour les images web scrapées et un autre pour les photos prises avec une caméra
web_images_folder = '/content/drive/MyDrive/RIMAI/by scarp1'
cam_images_folder = '/content/drive/MyDrive/RIMAI/Camera'

# Fonction pour renommer les images avec un identifiant unique incrémental
# et une partie fixe du nom (dans ce cas, "image_name")
def rename_images(folder_path, prefix):
    counter = 1
    for filename in os.listdir(folder_path):
        # Vérifier si le fichier est une image en se basant sur son extension
        if filename.lower().endswith(('.jpeg', '.jpg', '.png', '.gif')):
            # Séparer le nom du fichier de son extension
            base_name = os.path.splitext(filename)[0]
            file_extension = os.path.splitext(filename)[1]
            
            # Construire le nouveau nom de fichier
            # Format : prefix + numéro + partie fixe + extension
            # Exemple : web1_image_name.jpg ou cam1_image_name.png
            new_name = f"{prefix}{counter}_image_name{file_extension}"
            
            # Créer les chemins complets pour l'ancien et le nouveau fichier
            old_file = os.path.join(folder_path, filename)
            new_file = os.path.join(folder_path, new_name)
            
            # Renommer le fichier
            os.rename(old_file, new_file)
            print(f'Renamed {old_file} to {new_file}')
            
            # Incrémenter le compteur pour le prochain fichier
            counter += 1

# Appliquer la fonction de renommage aux images web
# Le préfixe 'web' est utilisé, donc les fichiers seront nommés web1_image_name.jpg, web2_image_name.png, etc.
rename_images(web_images_folder, 'web')

# Appliquer la fonction de renommage aux photos de caméra
# Le préfixe 'cam' est utilisé, donc les fichiers seront nommés cam1_image_name.jpg, cam2_image_name.png, etc.
rename_images(cam_images_folder, 'cam')